# libraries


In [107]:
#libraries and dirrectory 
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from scipy.optimize import curve_fit
import patientFunctions as ptfn
import seaborn as sns
import matplotlib.pyplot as plt
import re
import matplotlib.patches as mpatches


# directory

In [108]:
#location of original data
starting_directory = os.getcwd()


machine_directory = 'C:/Users/mcremer' #the C and path to the project folder
# machine_directory = 'C:/Users/maega' #when working from home machine
storage_directory = 'UFL Dropbox/Maegan Cremer/research-share/Maegan/Projects' #Local, HPG, or dropbox
project_directory = 'Cardiac-Amyloidosis-Multiple-Myeloma' #project folder
project_lv2_directory = '012 Processed Data/data-2024-06-05' #deeper part of project folder

path = os.path.join(machine_directory, storage_directory, 
                    project_directory, project_lv2_directory)

new_dirrectory = path

os.chdir(new_dirrectory)

In [109]:
#updates to directory management
machine_directory = 'C:/Users/mcremer' #the C and path to the project folder
# machine_directory = 'C:/Users/maega' #when working from home machine
storage_directory = 'UFL Dropbox/Maegan Cremer/research-share/Maegan/Projects' #Local, HPG, or dropbox
project_directory = 'Cardiac-Amyloidosis-Multiple-Myeloma' #project folder
project_lv2_directory = '015_Mistic2025' #deeper part of project folder

path = os.path.join(machine_directory, storage_directory, 
                    project_directory, project_lv2_directory)

# parent_dir = path
outputDir = path

## files and folders

In [110]:
folderNameHeader = "DF_SelectedFeats_"
folderNameCommon = ""
folderNameSuffix = "_20250416"

fileNameHeader_1 = "DF_SelectedFeats_"
# fileNameHeader_2 = "ALUnknownPts"
# fileNameCommon = "_Features"
fileNameSuffix = "_20250416"

folder_data_out = ptfn.makeFolderPathForData(parent_dir=outputDir, folderName_header=folderNameHeader, 
                                        folderName_common=folderNameCommon, folderName_suffix=folderNameSuffix)


folderNameOut = folder_data_out

# import patient data

In [111]:
#importing patient data as dictionaries of dataframes

#build a dictionary of patients and their values 
ptFiles = ptfn.GetListofPTfiles(new_dirrectory, '.xlsx', 'data')

ptDict = {}

#loop through the listOfPatients 
for file in ptFiles: 
    pt = ptfn.patient(new_dirrectory, file, 4)
    ptDict[pt.ptID] = pt

In [112]:
EchoTimeColumn ='time from diagnosis to first echo (months)'
MRITimeColumn = 'time from dagnosis to MRI (months)'
echoDates = {}
for pt in ptDict.keys():
    ptEchoData = ptDict[pt].echoData
    patientEchoDate = ptEchoData[EchoTimeColumn]
    if 'none' not in list(patientEchoDate):
        echoDates[pt] = patientEchoDate

#test to see if this still runs and if the data is correct

In [113]:
#list of patients by amyloid status 

listPositive = []
listNegative = []
listOther = []


list_pts_remove = ["K-01", "G-01", "J-01", "L-02"]
#remove list_pts_remove from ptDict
for pt in list_pts_remove:
    if pt in ptDict:
        del ptDict[pt]

#get list of positive patients 
for pt in ptDict:
    ptALStatus = ptDict[pt].amyloid_status
    if ptALStatus == 'yes':
        listPositive.append(pt)
    elif ptALStatus == 'no':
        listNegative.append(pt)
    else:
        listOther.append(pt)


In [114]:

listKnown = listPositive + listNegative

# import full feature list from DescFits
this feature list includes all features for the DescOnly, ParamsOnly, and DescFits models

In [115]:
os.chdir(path) #chaning directory to the location of our sheets

In [116]:
# full dataframe for the DescFits model
DescFitsFullFeatureList = pd.read_excel("DFsForSVM_20250416\DF_SVM_DescFits__20250416.xlsx")
DescFitsFullFeatureList.index = DescFitsFullFeatureList["DeID"]

<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:2: SyntaxWarning: invalid escape sequence '\D'
C:\Users\mcremer\AppData\Local\Temp\ipykernel_15872\98973414.py:2: SyntaxWarning: invalid escape sequence '\D'
  DescFitsFullFeatureList = pd.read_excel("DFsForSVM_20250416\DF_SVM_DescFits__20250416.xlsx")


In [117]:
#full dataframe for the EchoAdj model 
EchoAdjFullFeatureList = pd.read_excel("DFsForSVM_20250416\DF_SVM_EchoAdj__20250416.xlsx")
EchoAdjFullFeatureList.index = EchoAdjFullFeatureList["DeID"]

<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:2: SyntaxWarning: invalid escape sequence '\D'
C:\Users\mcremer\AppData\Local\Temp\ipykernel_15872\515290013.py:2: SyntaxWarning: invalid escape sequence '\D'
  EchoAdjFullFeatureList = pd.read_excel("DFsForSVM_20250416\DF_SVM_EchoAdj__20250416.xlsx")


In [118]:
#to get only the lab names used as stems of features
labsFiltered = pd.read_excel("DFsForSVM_20250416\SelectedFeatures_labs__20250416.xlsx")

<>:2: SyntaxWarning: invalid escape sequence '\S'
<>:2: SyntaxWarning: invalid escape sequence '\S'
C:\Users\mcremer\AppData\Local\Temp\ipykernel_15872\1132683709.py:2: SyntaxWarning: invalid escape sequence '\S'
  labsFiltered = pd.read_excel("DFsForSVM_20250416\SelectedFeatures_labs__20250416.xlsx")


# import the list of selected features for each model

In [119]:
selectedFeatures_allmodels_allfeatRed = pd.read_excel("SVM_model_comparison_2025-04-11.xlsx", sheet_name = "new wo counts features", header = [0,1]).drop([0], axis = 0)

In [120]:
## pull the selected feature lists for the models you are interested in

In [121]:
selectedFeatures_allmodels_allfeatRed

model data                                     DescFits  \
   feature reduction                                          0.1   
1                  1                                Chloride_mean   
2                  2                               Creatinine_min   
3                  3                               Hemoglobin_min   
4                  4                                BUN_linear_r2   
5                  5                           Monocyte_linear_r2   
6                  6                             Sodium_linear_r2   
7                  7                  NT Pro BNP_linear_intercept   
8                  8                                   BUN_exp_r2   
9                  9           Beta-2-Microglobulin_exp_intercept   
10                10                            Creatinine_exp_r2   
11                11                                 MCV_exp_coef   
12                12  Serum Electrophoresis Alpha 2_exp_intercept   
13                13                                          NaN   
14                14                                          NaN   
15                15                                          NaN   
16                16                                          NaN   
17                17                                          NaN   
18                18                                          NaN   
19                19                                          NaN   
20                20                                          NaN   
21                21                                          NaN   
22                22                                          NaN   
23                23                                          NaN   
24                24                                          NaN   
25                25                                          NaN   
26                26                                          NaN   
27                27                                          NaN   
28                28                                          NaN   
29                29                                          NaN   
30                30                                          NaN   
31                31                                          NaN   
32                32                                          NaN   
33                33                                          NaN   
34                34                                          NaN   
35                35                                          NaN   
36                36                                          NaN   
37                37                                          NaN   
38                38                                          NaN   
39                39                                          NaN   
40                40                                          NaN   
41               NaN                                          NaN   
42               NaN                                          NaN   

                                                  \
                                             0.2   
1                                            age   
2                                       BUN_mean   
3                                        BUN_min   
4                                        BUN_25%   
5                                        BUN_50%   
6                                  Chloride_mean   
7                                   Chloride_50%   
8                                   Chloride_75%   
9                                   Chloride_max   
10                                Creatinine_min   
11                                Creatinine_25%   
12                                Creatinine_50%   
13                                       HCT_max   
14                                Hemoglobin_max   
15                                    Sodium_max   
16                                 total CO2_min   
17                               NT P

In [122]:
list(selectedFeatures_allmodels_allfeatRed.columns)

[('model data', 'feature reduction'),
 ('DescFits', 0.1),
 ('DescFits', 0.2),
 ('DescFits', 0.25),
 ('DescFits', 0.3),
 ('EchoAdj', 0.1),
 ('EchoAdj', 0.2),
 ('EchoAdj', 0.25),
 ('DescOnly', 0.1),
 ('DescOnly', 0.2),
 ('DescOnly', 0.25),
 ('ParamsOnly', 0.1),
 ('ParamsOnly', 0.2),
 ('ParamsOnly', 0.25)]

In [123]:
featureRed = 0.25 #feature reduction value of interest

#pull selected features for the models with the given feature reduction
selectedFeats_025_allmodels = selectedFeatures_allmodels_allfeatRed.xs(featureRed, axis = 1, level = 1)

#list of models 
modelNames = list(selectedFeats_025_allmodels.columns)

# dataframes of only the features selected by each model

In [124]:
dict_df_selected = {}

for model in modelNames: 
    #get list of selected features 
    list_selectedFeatures = selectedFeats_025_allmodels[model].dropna()

    #check to make sure these features are in the model 
    test_listFeatures = [item for item in list_selectedFeatures if item in list(DescFitsFullFeatureList.columns)]
    if len(test_listFeatures) != 0:
        #slice the features dataframe per the selected features only
        df_selected = DescFitsFullFeatureList[list_selectedFeatures]

        #the index should by default be the DeID column from the original full feature list
        #adding the "CA_status_yes" back to enable filtering the patients as needed
        df_selected["CA_status_yes"]= DescFitsFullFeatureList["CA_status_yes"]
    else: 
        #the model uses the second feature dataframe, EchoAdj
        df_selected = EchoAdjFullFeatureList[list_selectedFeatures]
        
        #adding the CA status for filtering 
        df_selected["CA_status_yes"] = EchoAdjFullFeatureList["CA_status_yes"]

    #save that information somewhere 
    dict_df_selected[model] = df_selected

C:\Users\mcremer\AppData\Local\Temp\ipykernel_15872\3072580049.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected["CA_status_yes"]= DescFitsFullFeatureList["CA_status_yes"]
C:\Users\mcremer\AppData\Local\Temp\ipykernel_15872\3072580049.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected["CA_status_yes"] = EchoAdjFullFeatureList["CA_status_yes"]
C:\Users\mcremer\AppData\Local\Temp\ipykernel_15872\3072580049.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

## output data to sheets

In [125]:
#output data to sheets
for model in dict_df_selected.keys():
    data = dict_df_selected[model]

    #all patients, only selected features
    ptfn.outputToExcel(df_data= data.drop("CA_status_yes", axis = 1), 
                       fileName_header=fileNameHeader_1,
                       fileName_Common= model+"_"+str(featureRed).replace(".","")+"_", 
                       fileName_suffix= fileNameSuffix,
                       parent_dir=outputDir, folderName=folderNameOut,
                       sheetName= "allpts")
    
    #CA positive patients 
    ptfn.outputToExcel(df_data= data[data["CA_status_yes"]==1].drop("CA_status_yes", axis = 1), 
                        fileName_header=fileNameHeader_1,
                        fileName_Common= model+"_"+str(featureRed).replace(".","")+"_", 
                        fileName_suffix= fileNameSuffix,
                        parent_dir=outputDir, folderName=folderNameOut,
                        sheetName= "positive")
    #CA positive patients 
    ptfn.outputToExcel(df_data= data[data["CA_status_yes"]==0].drop("CA_status_yes", axis = 1), 
                        fileName_header=fileNameHeader_1,
                        fileName_Common= model+"_"+str(featureRed).replace(".","")+"_", 
                        fileName_suffix= fileNameSuffix,
                        parent_dir=outputDir, folderName=folderNameOut,
                        sheetName= "negative")